# EDA and Stakeholder Questions Notebook

In [4]:
# Import the required libraries
import pandas as pd
import numpy as np

import scipy.stats as scs

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

In [5]:
# Any Function Definitions Go HERE



In [7]:
# read in the processed data
cleaned_tweets = pd.read_csv('../data/cleaned_tweets.csv')
print(cleaned_tweets.shape)
cleaned_tweets.head(3)

(8699, 6)


,tweet_text,brand_or_product,detected_emotion,sentiment,brand,tweet_char_count
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,Neutral,Apple,127
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,Positive,Apple,139
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,Positive,Apple,79


## Question 1: What is the general breakdown for these Tweets?
### How many/what percentage Positive/Negative/Neutral?
### How many/what percentage Apple/Google/Unknown brand?


In [ ]:
# Q1 Insights/Recommendations:

## Question 2: Are the two companies represented equally in the labeled data?
### Percent Google? Percent Apple?  Out of all, out of all where not UNKNOWN?

In [ ]:
# Q2 Insights/Recommendations:

## Question 3: What insights can the data provide for specific products?
### Breakdown of brand_or_product and sentiment where not UNKNOWN
### Average char count for brand_or_product and sentiment where not UNKNOWN

In [ ]:
# Q3 Insights/Recommendations:

## Future Work